In [ ]:
!pip install pandas numpy silence_tensorflow tensorflow

In [1]:
import pandas as pd
import numpy as np
import os
from silence_tensorflow import silence_tensorflow
silence_tensorflow()
from tensorflow.keras.models import load_model

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 0번 GPU만 사용합니다.

In [2]:
model_1 = load_model('./weights/best_model_1.h5') # train에서 학습된 3개의 모델 구조와 가중치를 불러옵니다.
model_2 = load_model('./weights/best_model_2.h5')
model_3 = load_model('./weights/best_model_3.h5')

In [3]:
def apply_zero_padding(spectrogram, max_length):
    padding_length = max_length - spectrogram.shape[1]
    if padding_length > 0:
        return np.pad(spectrogram, ((0, 0), (0, padding_length)), 'constant')
    else:
        return spectrogram

In [4]:
submission_df = pd.read_csv('./data/anwer_sample.csv') # 제출 파일의 양식을 가져옵니다.

In [5]:
# 훈련 데이터 길이 중 가장 긴 길이를 불러옵니다.
npy_files = [f for f in os.listdir('./data/train_npy/') if f.endswith('.npy')] # 훈련 npy 파일이 들어있는 폴더 경로

spectrogram_shapes = {}
for file in npy_files:
    data = np.load(f"./data/train_npy/{file}") # 훈련 npy 파일이 들어있는 폴더 경로
    spectrogram_shapes[file] = data.shape

max_time_length = max(shape[1] for shape in spectrogram_shapes.values())

In [6]:
X_test = np.array([
    apply_zero_padding(np.load(f"./data/test_npy/{file}"), max_time_length) # 테스트 npy 파일이 들어있는 경로
    for file in submission_df['file']
])
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], X_test.shape[2], 1))

In [7]:
predictions_1 = model_1.predict(X_test) # 세 개의 모델을 모두 예측하고 예측 값을 평균을 내어 최종 예측을 결정합니다.
predictions_2 = model_2.predict(X_test)
predictions_3 = model_3.predict(X_test)
predictions = (predictions_1 + predictions_2 + predictions_3) / 3
predicted_labels = (predictions > 0.5).astype(int)
submission_df['label'] = predicted_labels
submission_df.to_csv('result.csv', index=False) # csv 파일이 저장될 경로

94/94 [==============================] - 1s 11ms/step
